In [1]:
import os
import random as rd
import glob
import numpy as np
import cv2

In [8]:
with_mask = glob.glob(r"D:\deep learning\mask\archive (4)\data\with_mask/*.*")
without_mask = glob.glob(r"D:\deep learning\mask\archive (4)\data\without_mask/*.*")

In [9]:
data = []
labels = []

for i in with_mask:
    img=cv2.imread(i)
    img=cv2.resize(img, (250,250))
    data.append(img)
    labels.append(0)
    
    
for i in without_mask:
    img=cv2.imread(i)
    img=cv2.resize(img, (250,250))
    data.append(img)
    labels.append(1)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
x_train,x_test,y_train,y_test=train_test_split(np.array(data),np.array(labels),test_size=0.2) 
# becoz list is there convert to array

In [12]:
x_train.shape  # x_train,no need to do preprocessing,already in crt shpe(no.of datapoints,width,height,depth)

(6042, 250, 250, 3)

In [13]:
from keras.utils import np_utils  # convert to categorical

In [14]:
y_trains=np_utils.to_categorical(y_train,2)
y_tests=np_utils.to_categorical(y_test,2)   # convert to categorical so that we can use softmax in final layer

In [15]:
y_trains[0]  # 1st image is without mask

array([1., 0.], dtype=float32)

# Directlty using pretrained model to implement transfer learning

In [19]:
from keras.applications.vgg16 import VGG16   # vgg16 train on colour image,convert grey to colour

In [21]:
# give wight ,dimension,whether model is trainale or not check
main=VGG16(include_top=False,weights="imagenet",input_shape=(250,250,3))   # include cnn layer without dense becoz to categorize our model,weight from imagenet

58889256/58889256 [==============================] - 620s 11us/step


In [ ]:
# downloading weights

In [22]:
main.trainable=False  # model not learning wight just take weight from imagenet,dense will update the weight

In [ ]:
#main.trainable=true- training do

In [23]:
main.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 250, 250, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 250, 250, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 250, 250, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 125, 125, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 125, 125, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 125, 125, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 62, 62, 128)       0     

In [25]:
from keras.models import Sequential
from keras.layers import Dense,Flatten

In [26]:
model=Sequential()
model.add(main)  # main add to model
model.add(Flatten())
model.add(Dense(80,activation="relu"))
model.add(Dense(2,activation="softmax"))  # softmax becoz one hot 

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 80)                2007120   
                                                                 
 dense_1 (Dense)             (None, 2)                 162       
                                                                 
Total params: 16,721,970
Trainable params: 2,007,282
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
#Trainable params- dense (Dense) + dense_1 (Dense)  ,train on 2,007,282 not on 14714688

In [28]:
model.compile(optimizer="rmsprop",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(x_train,y_trains,batch_size=100,epochs=1,validation_split=0.2)

In [ ]:
main=VGG16(include_top=False,weights="imagenet",input_shape=(250,250,3)) 
# by changing vgg16 to another pretrained model

In [ ]:
main.trainable=True

In [ ]:
main.summary()

In [ ]:
for layers in main.layers:
    if layers.name.startswith("block5_"):
        layers.trainable=True   # traing only last layer,freeze 1st 3 layers
    else:
        layers.trainable=False
        
    print(layers.name,layers.trainable)